## Initialisation and Training of ANN-1

In [ ]:
# importing necessary modules
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Input, Model
import tensorflow as tf

In [ ]:
# testing if GPU is active
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# connecting to google drive 
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/My Drive/content")

# reading Black-Scholes data
data = np.genfromtxt('bs_data.csv', delimiter=',')

# data prep

# inputs: converting spot price + strike price to moneyness (s/k)
inputs = data[:,:5] 
X = np.array([np.array([x[0]/x[1], x[2],x[3],x[4]])for x in inputs])

# outputs: scaling the prices byt the strike price (p/k)
y_calls = np.array(data[:,-2]) / data[:,1]
y_puts = np.array(data[:,-1]) / data[:,1]

# training dataset
X_train = X[:900000]
y_calls_train = y_calls[:900000]
y_puts_train = y_puts[:900000]

# validation dataset
X_val = X[900000:950000]
y_calls_val = y_calls[900000:950000]
y_puts_val = y_puts[900000:950000]

# testing dataset
X_test = X[950000:]
y_calls_test = y_calls[950000:]
y_puts_test = y_puts[950000:]

print(X_train.shape)
print(y_calls_train.shape)
print(y_puts_train.shape)
print(X_val.shape)
print(y_calls_val.shape)
print(y_puts_val.shape)
print(X_test.shape)
print(y_calls_test.shape)
print(y_puts_test.shape)

Mounted at /content/drive
(900000, 4)
(900000,)
(900000,)
(50000, 4)
(50000,)
(50000,)
(50000, 4)
(50000,)
(50000,)


In [ ]:
# model initialisation
inputs = Input(shape=(4,), name='input')
x = Dense(300, activation='relu', kernel_initializer='glorot_uniform')(inputs)
x = Dense(300, activation='relu', kernel_initializer='glorot_uniform')(x)
x = Dense(300, activation='relu', kernel_initializer='glorot_uniform')(x)
x = Dense(300, activation='relu', kernel_initializer='glorot_uniform')(x)
call = Dense(1, activation = 'relu', name='call_output')(x)
put = Dense(1, activation = 'relu', name='put_output')(x)

model = Model(inputs=inputs, outputs=[call, put])

opt = tf.keras.optimizers.Adam(learning_rate=1e-05)
model.compile(loss='mse', optimizer=opt)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 4)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 300)          1500        ['input[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 300)          90300       ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 300)          90300       ['dense_1[0][0]']                
                                                                                              

In [ ]:
# training model
model.fit(X_train, {'call_output': y_calls_train, 'put_output': y_puts_train},
                validation_data = (X_val, {'call_output': y_calls_val, 'put_output': y_puts_val}),
                epochs=200, batch_size=200)

Epoch 1/200
4500/4500 [==============================] - 12s 3ms/step - loss: 0.0156 - call_output_loss: 0.0121 - put_output_loss: 0.0034 - val_loss: 1.1926e-04 - val_call_output_loss: 6.6987e-05 - val_put_output_loss: 5.2277e-05
Epoch 2/200
4500/4500 [==============================] - 15s 3ms/step - loss: 3.4493e-05 - call_output_loss: 1.9807e-05 - put_output_loss: 1.4686e-05 - val_loss: 1.0142e-05 - val_call_output_loss: 5.7696e-06 - val_put_output_loss: 4.3727e-06
Epoch 3/200
4500/4500 [==============================] - 13s 3ms/step - loss: 6.7094e-06 - call_output_loss: 3.8893e-06 - put_output_loss: 2.8201e-06 - val_loss: 5.8675e-06 - val_call_output_loss: 4.0188e-06 - val_put_output_loss: 1.8487e-06
Epoch 4/200
4500/4500 [==============================] - 12s 3ms/step - loss: 3.3763e-06 - call_output_loss: 1.9535e-06 - put_output_loss: 1.4228e-06 - val_loss: 2.6836e-06 - val_call_output_loss: 1.5105e-06 - val_put_output_loss: 1.1731e-06
Epoch 5/200
4500/4500 [=====================

In [ ]:
# saving ANN-1
model.save('ANN-1')

INFO:tensorflow:Assets written to: ANN-1/assets


In [ ]:
# for re-loading of ANN-1
model = keras.models.load_model("ANN-1")